# Download and Clean Dataset

In [1]:
import pandas as pd
import numpy as np

## Let's download the data and read it into a pandas dataframe.

In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


## Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

## Let's check the dataset for any missing values.

In [4]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Split data into predictors and target

In [5]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

## Normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

In [6]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [7]:
n_cols = predictors_norm.shape[1] # number of predictors
n_cols

8

## import the libraries

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import keras

from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


# Part A Build a baseline model

## Randomly split the data into a training and test sets by holding 30% of the data for testing.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=42)

In [10]:
#build the regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu', ))
    model.add(Dense(10, activation='relu', ))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Train and test the model

In [11]:
model = regression_model()

Instructions for updating:
Colocations handled automatically by placer.


## Train the model with 50 epoachs

In [12]:
# fit the model
epochs = 50
model.fit(X_train, y_train, epochs=epochs, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
721/721 [==============================] - 1s 2ms/step - loss: 1581.3531
Epoch 2/50
721/721 [==============================] - 1s 754us/step - loss: 1563.1246
Epoch 3/50
721/721 [==============================] - 1s 709us/step - loss: 1538.0370
Epoch 4/50
721/721 [==============================] - 1s 725us/step - loss: 1498.5539
Epoch 5/50
721/721 [==============================] - 1s 745us/step - loss: 1429.2687
Epoch 6/50
721/721 [==============================] - 1s 776us/step - loss: 1314.6197
Epoch 7/50
721/721 [==============================] - 1s 744us/step - loss: 1146.4713
Epoch 8/50
721/721 [==============================] - 1s 721us/step - loss: 952.1031
Epoch 9/50
721/721 [==============================] - 1s 758us/step - loss: 775.0472
Epoch 10/50
721/721 [==============================] - 1s 716us/step - loss: 620.1406
Epoch 11/50
721/721 [==============================] - 1s 719us/step - loss: 481.8631
Epoch 12/5

## Evaluate the model using test data

In [13]:

loss_val = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)
loss_val


309/309 [==============================] - 0s 593us/step


122.69514894948422

In [14]:
#  compute the mean squared error between the predicted concrete strength and the actual concrete strength
mean_square_error = mean_squared_error(y_test, y_pred)
mean = np.mean(mean_square_error)
standard_deviation = np.std(mean_square_error)
print(mean, standard_deviation)

122.69515160705737 0.0


In [15]:
#Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors
total_mse = 50
epochs = 50
mean_squared_errors = []
for i in range(0, total_mse):
    #X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=i)
    model.fit(X_train, y_train, epochs=epochs, verbose=0)
    MSE = model.evaluate(X_test, y_test, verbose=0)
    print("MSE "+str(i+1)+": "+str(MSE))
    y_pred = model.predict(X_test)
    mean_square_error = mean_squared_error(y_test, y_pred)
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Below is the mean and standard deviation of " +str(total_mse) + " mean squared errors without normalized data. Total number of epochs for each training is: " +str(epochs) + "\n")
print("Mean: "+str(mean))
print("Standard Deviation: "+str(standard_deviation))

MSE 1: 93.48613156315578
MSE 2: 74.14437058828409
MSE 3: 53.64847905427507
MSE 4: 44.084219009745084
MSE 5: 39.33207804485432
MSE 6: 38.765261097633335
MSE 7: 37.52178772367706
MSE 8: 38.553240877910724
MSE 9: 36.21153221006918
MSE 10: 35.39173233007536
MSE 11: 34.77166920881055
MSE 12: 34.28968367221672
MSE 13: 34.69246522971341
MSE 14: 35.729949599330865
MSE 15: 33.94306302610725
MSE 16: 34.39456410083956
MSE 17: 34.850082483878026
MSE 18: 33.745372154951866
MSE 19: 34.657362348439236
MSE 20: 34.52531176940523
MSE 21: 34.8046997712268
MSE 22: 36.677861611819964
MSE 23: 36.99336019843142
MSE 24: 34.64361538316054
MSE 25: 34.998609339149255
MSE 26: 34.42018234845504
MSE 27: 35.046757904842835
MSE 28: 35.28670566521802
MSE 29: 36.140260770482925
MSE 30: 35.12285405995391
MSE 31: 35.53272231265565
MSE 32: 35.082885760705445
MSE 33: 35.7157764002729
MSE 34: 35.958799880685156
MSE 35: 34.83397344941074
MSE 36: 35.38072471248294
MSE 37: 35.699634712490834
MSE 38: 36.22664443574677
MSE 39: 3

# Compared with step B, when there are three hidden layers, the mean is smaller and the standard deviation is smaller. So more hidden layers model is better and more epochs can train the model well